<a href="https://colab.research.google.com/github/basavarajmullur/Spring-Boot-JdbcTemplate/blob/master/notebooks/quick_start_with_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install -q transformers accelerate bitsandbytes
!pip install -q fastapi uvicorn pyngrok pillow


## Setup

To complete this tutorial, you'll need to have a runtime with [sufficient resources](https://ai.google.dev/gemma/docs/core#sizes) to run the MedGemma model.

You can try out MedGemma 4B for free in Google Colab using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

**Note**: To run the demo with MedGemma 27B in Google Colab, you will need a runtime with an A100 GPU.

### Get access to MedGemma

Before you get started, make sure that you have access to MedGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [MedGemma model page](https://huggingface.co/google/medgemma-1.5-4b-it) and accept the usage conditions.

### Step 1: Authenticate with Hugging Face


In [1]:
from huggingface_hub import login
login()

### Step 2: Install dependencies

In [16]:
!pip install -q \
  fastapi \
  uvicorn \
  transformers \
  accelerate \
  bitsandbytes \
  pillow==10.4.0 \
  torch torchvision \





## Step 3: Load MedGemma

In [17]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

MODEL_ID = "google/medgemma-4b-it"

processor = AutoProcessor.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()
print("✅ MedGemma loaded")


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

✅ MedGemma loaded


## Step 4: Install cloudflared

In [18]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

## Step 5: Install cloudflared

In [19]:
from fastapi import FastAPI, UploadFile, Form
from PIL import Image
import io
import json

app = FastAPI(title="ClinIQ – MedGemma API")

SYSTEM_PROMPT = """
You are a clinical decision support assistant.

Rules:
- Do NOT provide diagnoses
- Use observational language only
- Explicitly state uncertainty
- Phrase findings for clinicians
- Avoid prescriptive advice

Respond ONLY with valid JSON.

JSON schema:
{
  "observations": [],
  "possible_interpretations": [],
  "uncertainty_notes": "",
  "recommend_next_steps": []
}
"""

@app.post("/analyze")
async def analyze(
    image: UploadFile,
    clinical_question: str = Form(...)
):
    img_bytes = await image.read()
    img = Image.open(io.BytesIO(img_bytes)).convert("RGB")

    full_prompt = f"""
Return ONLY valid JSON.

{SYSTEM_PROMPT}

Clinical question:
{clinical_question}
"""

    inputs = processor(
        images=img,
        text=full_prompt,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.4
        )

    text = processor.decode(output_ids[0], skip_special_tokens=True)

    # enforce JSON safety
    try:
        return json.loads(text)
    except Exception:
        return {
            "error": "Model did not return valid JSON",
            "raw_output": text
        }



## Step 6: Run FastAPI server



In [20]:
import uvicorn
import threading

threading.Thread(
    target=uvicorn.run,
    kwargs={
        "app": app,
        "host": "0.0.0.0",
        "port": 8000
    },
    daemon=True
).start()

print("🚀 FastAPI running on port 8000")


🚀 FastAPI running on port 8000


## Step 7 Expose via Cloudflare Tunnel

In [24]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

import subprocess
import threading
import re

def cloudflare():
    process = subprocess.Popen(
        ["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8000"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )

    for line in process.stdout:
        print(line, end="")

        # Extract public URL
        match = re.search(r"https://[a-zA-Z0-9\-]+\.trycloudflare\.com", line)
        if match:
            print(f"\n🌍 Public endpoint: {match.group(0)}\n")

threading.Thread(target=cloudflare, daemon=True).start()

